In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import psycopg2

In [2]:
df = pd.read_csv("data/mental-illnesses-prevalence.csv",  encoding="utf-8")

In [3]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 8 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Entity                                                                             6420 non-null   object 
 1   Code                                                                               6150 non-null   object 
 2   Year                                                                               6420 non-null   int64  
 3   Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized  6420 non-null   float64
 4   Depressive disorders (share of population) - Sex: Both - Age: Age-standardized     6420 non-null   float64
 5   Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized        6420 non-null   float6

In [4]:
try:
    # Read a specific sheet by name
    working_hours = pd.read_excel("/Users/jamie/neuefische/MindScape/data/pwt1001.xlsx", sheet_name="Data")
    
    # Or read a specific sheet by index (e.g., the first sheet)
    # df = pd.read_excel("/path/to/your/file.xlsx", sheet_name=0)
    
    print(working_hours.head())
except Exception as e:
    print(f"An error occurred: {e}") 

  countrycode country   currency_unit  year  rgdpe  rgdpo  pop  emp  avh  hc  \
0         ABW   Aruba  Aruban Guilder  1950    NaN    NaN  NaN  NaN  NaN NaN   
1         ABW   Aruba  Aruban Guilder  1951    NaN    NaN  NaN  NaN  NaN NaN   
2         ABW   Aruba  Aruban Guilder  1952    NaN    NaN  NaN  NaN  NaN NaN   
3         ABW   Aruba  Aruban Guilder  1953    NaN    NaN  NaN  NaN  NaN NaN   
4         ABW   Aruba  Aruban Guilder  1954    NaN    NaN  NaN  NaN  NaN NaN   

   ...  csh_x  csh_m  csh_r  pl_c  pl_i  pl_g  pl_x  pl_m  pl_n  pl_k  
0  ...    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1  ...    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2  ...    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3  ...    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4  ...    NaN    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 52 columns]


In [5]:
working_hours.columns

Index(['countrycode', 'country', 'currency_unit', 'year', 'rgdpe', 'rgdpo',
       'pop', 'emp', 'avh', 'hc', 'ccon', 'cda', 'cgdpe', 'cgdpo', 'cn', 'ck',
       'ctfp', 'cwtfp', 'rgdpna', 'rconna', 'rdana', 'rnna', 'rkna', 'rtfpna',
       'rwtfpna', 'labsh', 'irr', 'delta', 'xr', 'pl_con', 'pl_da', 'pl_gdpo',
       'i_cig', 'i_xm', 'i_xr', 'i_outlier', 'i_irr', 'cor_exp', 'statcap',
       'csh_c', 'csh_i', 'csh_g', 'csh_x', 'csh_m', 'csh_r', 'pl_c', 'pl_i',
       'pl_g', 'pl_x', 'pl_m', 'pl_n', 'pl_k'],
      dtype='object')

In [6]:
working_hours_cleaned = working_hours.dropna(subset=['avh'])


In [7]:
working_hours_cleaned 

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_x,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k
350,ARG,Argentina,Argentine Peso,1950,50303.953125,50108.761719,17.091824,6.608833,2034.000000,1.816503,...,0.122540,-0.060423,-0.012842,0.193067,0.422730,0.273050,0.117665,0.107814,0.304496,NaN
351,ARG,Argentina,Argentine Peso,1951,52677.964844,51339.167969,17.457578,6.713252,2037.866753,1.830769,...,0.100497,-0.074776,-0.016469,0.213862,0.366893,0.277869,0.142259,0.118755,0.275817,NaN
352,ARG,Argentina,Argentine Peso,1952,47677.207031,46855.101562,17.815967,6.819321,2041.740856,1.845147,...,0.077847,-0.058380,-0.033148,0.229605,0.366508,0.296447,0.126297,0.130694,0.279382,NaN
353,ARG,Argentina,Argentine Peso,1953,49529.671875,49917.148438,18.168618,6.927065,2045.622325,1.859638,...,0.106709,-0.041785,-0.022161,0.230860,0.370191,0.315614,0.110863,0.102362,0.278423,NaN
354,ARG,Argentina,Argentine Peso,1954,51835.375000,52254.769531,18.517168,7.036511,2049.511172,1.874243,...,0.108594,-0.051525,-0.029199,0.229684,0.381629,0.327768,0.094007,0.093510,0.285159,8.819025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12664,ZAF,South Africa,Rand,2014,705391.000000,710668.312500,54.544186,16.992683,2209.091467,2.675951,...,0.185648,-0.218741,0.013825,0.491612,0.500575,0.515295,0.697915,0.652995,0.329836,0.624894
12666,ZAF,South Africa,Rand,2016,722946.187500,713601.625000,56.207646,17.984968,2218.455068,2.762902,...,0.170088,-0.179245,0.005021,0.404804,0.445430,0.406019,0.628334,0.590239,0.275335,0.487648
12667,ZAF,South Africa,Rand,2017,738072.875000,726021.000000,57.009756,18.302843,2197.033263,2.809246,...,0.190417,-0.192079,-0.003146,0.465375,0.522530,0.454620,0.638229,0.595804,0.319646,0.566110
12668,ZAF,South Africa,Rand,2018,743338.562500,732308.187500,57.792518,18.686357,2191.363362,2.857641,...,0.193246,-0.203486,0.005634,0.488052,0.546290,0.478244,0.661046,0.620808,0.332931,0.563239


In [8]:
print(len(working_hours_cleaned['country'].unique())) 

69


In [9]:
working_hours = working_hours_cleaned[['country', 'year', 'avh']] 

In [10]:
working_hours.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 3492 entries, 350 to 12669
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  3492 non-null   object 
 1   year     3492 non-null   int64  
 2   avh      3492 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 109.1+ KB


In [11]:
working_hours.head() 

,country,year,avh
350,Argentina,1950,2034.000000
351,Argentina,1951,2037.866753
352,Argentina,1952,2041.740856
353,Argentina,1953,2045.622325
354,Argentina,1954,2049.511172


In [12]:
df.rename(columns={'Entity': 'country', 'Year': 'year', 'Code': 'code', 'Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized': 'pct_schizophrenia', 'Depressive disorders (share of population) - Sex: Both - Age: Age-standardized': 'pct_depression', 'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized': 'pct_anxiety', 'Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized': 'pct_bipolar', 'Eating disorders (share of population) - Sex: Both - Age: Age-standardized':'pct_eating_disorder' }, inplace=True) 

In [13]:
df_combined = pd.merge(df, working_hours, on=['country', 'year'], how='left')

In [59]:
df_combined.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              6420 non-null   object 
 1   code                 6150 non-null   object 
 2   year                 6420 non-null   int64  
 3   pct_schizophrenia    6420 non-null   float64
 4   pct_depression       6420 non-null   float64
 5   pct_anxiety          6420 non-null   float64
 6   pct_bipolar          6420 non-null   float64
 7   pct_eating_disorder  6420 non-null   float64
 8   avh                  1785 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 451.5+ KB


In [15]:
population = pd.read_csv('/Users/jamie/neuefische/MindScape/data/API_SP.POP.TOTL_DS2_en_csv_v2_85.csv', delimiter=',', encoding='utf-8', on_bad_lines='skip', header=2) 

In [16]:
population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,...,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130692579.0,134169237.0,137835590.0,141630546.0,145605995.0,149742351.0,...,583651101.0,600008424.0,616377605.0,632746570.0,649757148.0,667242986.0,685112979.0,702977106.0,720859132.0,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,...,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0,41128771.0,NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,97256290.0,99314028.0,101445032.0,103667517.0,105959979.0,108336203.0,...,397855507.0,408690375.0,419778384.0,431138704.0,442646825.0,454306063.0,466189102.0,478185907.0,490330870.0,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,5357195.0,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,...,27128337.0,28127721.0,29154746.0,30208628.0,31273533.0,32353588.0,33428486.0,34503774.0,35588987.0,NaN


In [17]:
pop_long = pd.melt(population, 
                  id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], 
                  var_name='year', 
                  value_name='value') 

In [49]:
pop_sorted = pop_long.sort_values(by='Country Name', ascending=False)

In [50]:
pop_sorted.dropna(subset=['value'], inplace = True) 

In [51]:
pop_sorted.rename(columns= {'Country Name': 'country', 'Country Code': 'code'}, inplace = True) 

In [52]:
pop_sorted.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 16665 entries, 531 to 2130
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         16665 non-null  object 
 1   code            16665 non-null  object 
 2   Indicator Name  16665 non-null  object 
 3   Indicator Code  16665 non-null  object 
 4   year            16665 non-null  object 
 5   value           16665 non-null  float64
dtypes: float64(1), object(5)
memory usage: 911.4+ KB


In [55]:
df_master = pd.merge(df_combined, pop_sorted, on=['code', 'year'], how='left') 

In [67]:
print(df_combined['year'].dtype)
print(pop_sorted['year'].dtype)


int64
int64


In [54]:
pop_sorted['year'] = pop_sorted['year'].astype(int)

In [56]:
df_master.rename(columns={'country_x': 'country', 'value': 'population'}, inplace = True)

In [57]:
df_master.drop(columns=['Indicator Name', 'country_y', 'Indicator Code'], inplace=True)

In [58]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              6420 non-null   object 
 1   code                 6150 non-null   object 
 2   year                 6420 non-null   int64  
 3   pct_schizophrenia    6420 non-null   float64
 4   pct_depression       6420 non-null   float64
 5   pct_anxiety          6420 non-null   float64
 6   pct_bipolar          6420 non-null   float64
 7   pct_eating_disorder  6420 non-null   float64
 8   avh                  1785 non-null   float64
 9   population           6000 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 501.7+ KB


In [68]:
# Set display option to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 

In [64]:
regions = df_master[df_master['code'].isnull()]
regions 

,country,code,year,pct_schizophrenia,pct_depression,pct_anxiety,pct_bipolar,pct_eating_disorder,avh,population
30,Africa (IHME GBD),NaN,1990,0.219527,4.602806,3.696839,0.607027,0.111027,NaN,NaN
31,Africa (IHME GBD),NaN,1991,0.219559,4.598041,3.695416,0.607090,0.110425,NaN,NaN
32,Africa (IHME GBD),NaN,1992,0.219579,4.593013,3.693819,0.607127,0.109845,NaN,NaN
33,Africa (IHME GBD),NaN,1993,0.219583,4.588568,3.692097,0.607134,0.109305,NaN,NaN
34,Africa (IHME GBD),NaN,1994,0.219556,4.586263,3.690115,0.607082,0.108813,NaN,NaN
35,Africa (IHME GBD),NaN,1995,0.219513,4.585838,3.687944,0.606987,0.108413,NaN,NaN
36,Africa (IHME GBD),NaN,1996,0.219426,4.593121,3.680501,0.606898,0.108053,NaN,NaN
37,Africa (IHME GBD),NaN,1997,0.219292,4.609175,3.665615,0.606820,0.107740,NaN,NaN
38,Africa (IHME GBD),NaN,1998,0.219151,4.627353,3.648328,0.606740,0.107471,NaN,NaN
39,Africa (IHME GBD),NaN,1999,0.219040,4.642278,3.633697,0.606659,0.107308,NaN,NaN


In [ ]:
# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [ ]:
regions = df_master[df_master['code'].isnull()]
regions 

In [65]:
df_master.dropna(subset='code', inplace = True)

In [66]:
df_master.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 6150 entries, 0 to 6419
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              6150 non-null   object 
 1   code                 6150 non-null   object 
 2   year                 6150 non-null   int64  
 3   pct_schizophrenia    6150 non-null   float64
 4   pct_depression       6150 non-null   float64
 5   pct_anxiety          6150 non-null   float64
 6   pct_bipolar          6150 non-null   float64
 7   pct_eating_disorder  6150 non-null   float64
 8   avh                  1785 non-null   float64
 9   population           6000 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 528.5+ KB


In [21]:
import csv

delimiters = [',', ';', '\t']  # Add other delimiters as needed

for delimiter in delimiters:
    try:
        with open('/Users/jamie/neuefische/MindScape/data/API_SP.POP.TOTL_DS2_en_csv_v2_85.csv', 'r', encoding='utf-8') as file:
            csv_reader = csv.reader(file, delimiter=delimiter)
            rows = list(csv_reader) 
        print(f"Delimiter '{delimiter}' works!") 
        break
    except Exception as e:
        print(f"Delimiter '{delimiter}' doesn't work: {e}")


Delimiter ',' works!


In [69]:
from sql_functions import get_engine, get_sql_config, get_api, get_dataframe, push_to_sql
sql_config = get_sql_config() 

# sql_query = f'select * from {schema}.flights limit 5;'

In [70]:
schema = 'cgn_analytics_24_1'
engine = get_engine()
table_name = 'mindscape_mhealth_vs_whours'

push_to_sql(df_master, table_name, engine, schema) 

'The mindscape_mhealth_vs_whours table was imported successfully.'